Aprendizagem por Reforço com Q-Learning

Entendimento do Ambiente

In [82]:
import gym
import random
import numpy as np
from time import sleep

In [53]:
env = gym.make('Taxi-v3').env

In [54]:
# env.render()

In [55]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [56]:
# 0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup, 5 = dropoff
print(env.action_space)

Discrete(6)


In [57]:
# 4 dsetinos
print(env.observation_space)

Discrete(500)


In [58]:
len(env.P)

500

In [59]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

Treinamento

In [60]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape

(500, 6)

In [74]:
# 1-10% 3-90%
# exploration / exploitation
# 0 = south 1 = north 2 = east 3 = west 4 = pickup 5 = dropoff
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(100000):
  estado = env.reset()

  penalidades, recompensa = 0, 0
  done = False
  while not done:
    # Exploração
    if random.uniform(0, 1) < epsilon:
      acao = env.action_space.sample()
    # Exploitation
    else:
      acao = np.argmax(q_table[estado])

    proximo_estado, recompensa, done, info = env.step(acao)

    q_antigo = q_table[estado, acao]
    proximo_maximo = np.max(q_table[proximo_estado])

    q_novo = (1 - alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
    q_table[estado, acao] = q_novo

    if recompensa == -10:
      penalidades += 1

    estado = proximo_estado

  if i % 100 == 0:
    clear_output(wait=True)
    print('Episódio: ', i)

print('Treinamento concluído')

Episódio:  99900
Treinamento concluído


In [75]:
q_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -2.41837066,  -2.3639511 ,  -2.41837066,  -2.3639511 ,
         -2.27325184, -11.3639511 ],
       [ -1.870144  ,  -1.45024   ,  -1.870144  ,  -1.45024   ,
         -0.7504    , -10.45024   ],
       ...,
       [ -1.04525712,   0.416     ,  -0.89586219,  -1.27781499,
         -5.15420151,  -6.09602988],
       [ -2.18977562,  -2.1220864 ,  -2.19041906,  -2.1220864 ,
         -8.25826072,  -5.66337427],
       [  4.23572361,   1.8863539 ,   4.27406063,  11.        ,
         -3.16668834,  -2.64685539]])

In [76]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [77]:
env.encode(3, 4, 2, 0)

388

In [78]:
q_table[388]

array([ -2.48041796,  -2.4510224 ,  -2.46973686,  -2.4510224 ,
       -11.32560378, -10.30035275])

Avaliação

In [80]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
    estado = env.reset()
    penalidades, recompesa = 0, 0
    done = False
    while not done:
        acao = np.argmax(q_table[estado])
        estado, recompensa, done, info = env.step(acao)

    if recompensa == -10:
        penalidades += 1

        frames.append({
            'frame': env.render(mode='ansi'),
            'state': estado,
            'action': acao,
            'reward': recompensa        
        })

    total_penalidades += penalidades
print('Episódios', episodios)
print('Penalidades', total_penalidades)

Episódios 50
Penalidades 0


In [83]:
for frame in frames:
    clear_output(wait=True)
    print(frame['frame'])
    print('Estado', frame['state'])
    print('Acão', frame['action'])
    print('Recompensa', frame['reward'])
    sleep(.1)